# Computing temperature-dependent intrinsic parameters

In this notebook we show how we can get temperature dependent Ms, A, and K by running spindynamics simulations for a range of temperatures and then fitting the Kuzmin equation to the resulting M(T). From the fit we can get A(T) and K(T).

In [1]:
import mammos_analysis
import matplotlib.pyplot as plt
from mammos_spindynamics import uppasd

## Creating the simulation object

First, we need to create a simulation object. We assume that we already have three files `posfile`, `momfile` and `jfile` in our current working directory and that they are in the form required for UppASD. We will not create `inpsd.dat` manually but instead rely on the simulation object. We can pass all required options for the input file as arguments when creating the simulation object or when calling one of its functions. Here, we will pass most arguments at creation time and only temperature values at runtime.

We take CrNiP as an example for this notebook.

In [2]:
sim = uppasd.Simulation(
    cell=[(5.058, -2.92, 0.0), (0.0, 5.84, 0.0), (0.0, 0.0, 3.51)],
    ncell=[25, 25, 25],
    BC=["P", "P", "P"],
    alat=1.00e-10,
    Initmag=3,
    posfiletype="D",
    mode="M",
    momfile="uppasd_data/momfile",
    posfile="uppasd_data/posfile",
    exchange="uppasd_data/jfile",
    mcnstep=1000,
)

To see a list of all available arguments (a subset of what UppASD supports) and information whether they are required and their default value (if any), use:

In [ ]:
uppasd.Simulation.defaults

## Running a temperature sweep

We now want to run simulations for a series of temperatures between 10 and 500 Kelvin, in steps of 50 Kelvin. We can use the `range` function to get all temperatures. The simulation object provides a method to automatically run all simulations sequentially. We can tell it to use the restart file of the previous calculation as starting point for the next simulation. For the first simulation it will use `Initmag=1`, which we have set above.

In [3]:
sim.temperature_sweep(
    T=range(10, 500, 50), restart_with_previous=True, out="CrNiP", description="Anything the user wants to record"
)

 --------------------------------------------------------------
             __  __          ___   _______    ____  ___        
            / / / /__  ___  / _ | / __/ _ \  / __/ / _ \       
           / /_/ / _ \/ _ \/ __ |_\ \/ // / / _ \_/ // /       
           \____/ .__/ .__/_/ |_/___/____/  \___(_)___/        
               /_/  /_/                                        
 --------------------------------------------------------------
                https://github.com/UppASD/UppASD               
 --------------------------------------------------------------
              Division of Materials Theory                     
              Department of Physics and Astronomy              
              Uppsala University                               
              Sweden                                           
 ---------------------Production-version-----------------------
 Git revision: v6.0.2-136-g553a
 --------------------------------------------------------------
 RNG: Me

TemperatureSweepData('CrNiP/temperature_sweep-0')

On disk, this call will create the following directory and file structure:

```
CrNiP/
    temperature_sweep-0/
        run-0/
            jfile
            posfile
            inpsd.dat
            cumulants.CrNiP.out
            ...
            mammos_spindynamics_info.{yaml,toml}
        run-1/
            ...
        run-2/
            ...
        ...
        mammos_spindynamics_info.{yaml,toml}
        M(T)
```

There is generally no need to access any files on the system by hand. Instead we can use the data analysis capabilities coming with `mammos_spindynamics`.

## Analyzing simulation output

First, we create a data object, which we point to the top-level output directory `CrNiP`. The object recursively finds all files in `CrNiP` and provides access to the simulation results from within Python.

In [2]:
data = uppasd.read("CrNiP")
data

MammosUppasdData('CrNiP')

In [4]:
data.info()

,id,description,time_elapsed,T
0,temperature_sweep-0,Anything the user wants to record,0:00:01.362103,"[10, 60, 110, 160, 210, 260, 310, 360, 410, 460]"


In [6]:
uppasd.MammosUppasdData("CrNiP")

MammosUppasdData('CrNiP')

We only have one set of outputs in this example. We can acces it as follows:

In [10]:
sweep_data = data[0]
sweep_data

TemperatureSweepData('CrNiP/temperature_sweep-0')

In [8]:
uppasd.TemperatureSweepData("CrNiP/temperature_sweep-0")

TemperatureSweepData('CrNiP/temperature_sweep-0')

In [9]:
uppasd.read("CrNiP/temperature_sweep-0")

TemperatureSweepData('CrNiP/temperature_sweep-0')

The object allows us to access the array-like simulation output and the temperatures:

In [11]:
sweep_data.T

Entity(ontology_label='ThermodynamicTemperature', value=array([ 10.,  60., 110., 160., 210., 260., 310., 360., 410., 460.]), unit='K')

In [12]:
sweep_data.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), unit='A / m')

We can also access the individual runs:

In [13]:
run_10K = sweep_data.sel(T=10)
run_10K

RunData('CrNiP/temperature_sweep-0/run-0')

In [14]:
uppasd.RunData("CrNiP/temperature_sweep-0/run-0")

RunData('CrNiP/temperature_sweep-0/run-0')

In [15]:
uppasd.read("CrNiP/temperature_sweep-0/run-0")

RunData('CrNiP/temperature_sweep-0/run-0')

In [16]:
run_10K.metadata

{'UppASD_version': 'v6.0.2',
 'description': '',
 'index': 0,
 'mammos_spindynamics_version': '0.2.4',
 'mode': 'run',
 'time_elapsed': '0:00:00.126262',
 'time_end': '2025-12-09T22:17:59+01:00',
 'time_start': '2025-12-09T22:17:59+01:00'}

In [17]:
run_10K.parameters

{'T': 10}

In [18]:
run_10K.Ms

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(nan), unit='A / m')

In [19]:
run_10K.restart_file

PosixPath('CrNiP/temperature_sweep-0/run-0/restart._UppASD_.out')

In [22]:
run_10K.describe(
    include_elapsed_time=True,
    include_index=False,
    include_parameters=True,
    include_description=True,
)

{'id': 'run-0', 'description': '', 'time_elapsed': '0:00:00.126262', 'T': 10}

## Kuzmin fit

We can now pass the data from the temperature sweep to the Kuzmin function in `mammos_analysis`:

In [ ]:
kuzmin_result = mammos_analysis.kuzmin.kuzmin_properties(Ms=sweep_data.Ms, T=sweep_data.T)

The kuzmin result object has a method to plot the temperature dependent parameters:

In [ ]:
kuzmin_result.plot()

We can also compare against our simulated values directly:

In [ ]:
kuzmin_result.Ms.plot()
plt.plot(range(10, 500, 50), sweep_data.Ms.q, "o")

We could use results from the Kuzmin fit as inputs for a micromagnetic simulation, e.g. to access parameters at 300 Kelvin (for which we did not run a simulation) we can use:

In [ ]:
kuzmin_result.Ms(300)

In [ ]:
kuzmin_result.A(300)